# Validate & Create Test Scenarios

**Source:** https://confluence.hl7.org/pages/viewpage.action?pageId=204276132

**Run this notebook in the cloud:**

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/beda-software/au-core-aidbox/main?labpath=client%2FValidate+%26+Create+Test+Scenarios.ipynb)

In [ ]:
from fhirpy import SyncFHIRClient
from pprint import pprint
from fhirpy.base.exceptions import OperationOutcome
import uuid

In [ ]:
SOURCE_BASE_URL = "https://au-core.beda.software/fhir/"

In [ ]:
client = SyncFHIRClient(SOURCE_BASE_URL)

## 1. Patient
### 1.1 $validate Patient
**Action:**  Validate the content of a FHIR formatted Patient resource

In [ ]:
valid_patient_data = {
  "resourceType": "Patient",
  "id": str(uuid.uuid4()),
  "identifier": [
    {
      "use": "usual",
      "type": {
        "coding": [
          {
            "system": "http://terminology.hl7.org/CodeSystem/v2-0203",
            "code": "MR",
            "display": "Medical Record Number"
          }
        ]
      },
      "system": "https://www.hospital.org/patients",
      "value": "123456"
    }
  ],
  "active": True,
  "name": [
    {
      "use": "official",
      "family": "Doe",
      "given": [
        "John",
        "A."
      ]
    }
  ],
  "telecom": [
    {
      "system": "phone",
      "value": "555-1234",
      "use": "home"
    },
    {
      "system": "email",
      "value": "john.doe@example.com",
      "use": "home"
    }
  ],
  "gender": "male",
  "birthDate": "1980-01-01",
  "address": [
    {
      "use": "home",
      "line": [
        "123 Main St",
        "Apartment 4"
      ],
      "city": "Anytown",
      "state": "CA",
      "postalCode": "90210",
      "country": "USA"
    }
  ],
  "maritalStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/v3-MaritalStatus",
        "code": "M",
        "display": "Married"
      }
    ],
    "text": "Married"
  },
  "contact": [
    {
      "relationship": [
        {
          "coding": [
            {
              "system": "http://terminology.hl7.org/CodeSystem/v2-0131",
              "code": "N",
              "display": "Next of Kin"
            }
          ]
        }
      ],
      "name": {
        "family": "Doe",
        "given": [
          "Emily"
        ]
      },
      "telecom": [
        {
          "system": "phone",
          "value": "555-6789",
          "use": "home"
        }
      ],
      "address": {
        "line": [
          "123 Main St"
        ],
        "city": "Anytown",
        "state": "CA",
        "postalCode": "90210",
        "country": "USA"
      },
      "gender": "female",
      "period": {
        "start": "2019-01-01"
      }
    }
  ],
  "communication": [
    {
      "language": {
        "coding": [
          {
            "system": "urn:ietf:bcp:47",
            "code": "en-US",
            "display": "English (United States)"
          }
        ],
        "text": "English"
      },
      "preferred": True
    }
  ]
}

invalid_patient_data = {"birthDate": "test"}

try:
    client.resource('Patient', **valid_patient_data) \
        .is_valid(raise_exception=True)
except OperationOutcome as e:
    pprint('Error: {}'.format(e))

### 1.2 create Patient
**Action:** FHIR client creates a new patient and save to FHIR server.

In [ ]:
patient = client.resource("Patient", **valid_patient_data)
patient.create()

In [ ]:
example_patient = client.reference('Patient', valid_patient_data["id"]).to_resource()
pprint(example_patient.serialize())

## 2. Condition
### 2.1 $validate Condition
**Action:**  Validate the content of a FHIR formatted Condition resource

In [ ]:
valid_condition_data = {
  "resourceType": "Condition",
  "id": str(uuid.uuid4()),
  "clinicalStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "active",
        "display": "Active"
      }
    ]
  },
  "verificationStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-ver-status",
        "code": "confirmed",
        "display": "Confirmed"
      }
    ]
  },
  "category": [
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/condition-category",
          "code": "encounter-diagnosis",
          "display": "Encounter Diagnosis"
        }
      ]
    }
  ],
  "severity": {
    "coding": [
      {
        "system": "http://snomed.info/sct",
        "code": "6736007",
        "display": "Moderate"
      }
    ]
  },
  "code": {
    "coding": [
      {
        "system": "http://snomed.info/sct",
        "code": "123456",
        "display": "Hypertension"
      }
    ],
    "text": "Hypertension"
  },
  "onsetDateTime": "2021-01-01",
  "recordedDate": "2021-01-10",
  "subject": {
    "reference": "Patient/wang-li",
    "display": "Wang Li"
  },
}

invalid_condition_data = {}

try:
    client.resource('Condition', **valid_condition_data) \
        .is_valid(raise_exception=True)
except OperationOutcome as e:
    pprint('Error: {}'.format(e))

### 2.2 create Condition
**Action:** FHIR client creates a new Condition instance for a patient and save to FHIR server.

In [ ]:
condition = client.resource("Condition", **valid_condition_data)
condition.create()

In [ ]:
example_condition = client.reference('Condition', valid_condition_data["id"]).to_resource()
pprint(example_condition.serialize())

## 3. AllergyIntolerance
### 3.1 $validate AllergyIntolerance
**Action:**  Validate the content of a FHIR formatted resource

In [ ]:
valid_allergy_data = {
  "resourceType": "AllergyIntolerance",
  "id": str(uuid.uuid4()),
  "clinicalStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
        "code": "active",
        "display": "Active"
      }
    ]
  },
  "verificationStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-verification",
        "code": "confirmed",
        "display": "Confirmed"
      }
    ]
  },
  "type": "allergy",
  "category": ["food"],
  "criticality": "high",
  "code": {
    "coding": [
      {
        "system": "http://snomed.info/sct",
        "code": "227493005",
        "display": "Peanut Allergy"
      }
    ]
  },
  "patient": {
    "reference": "Patient/wang-li",
    "display": "Wang Li"
  },
  "onsetDateTime": "2010-06-12",
  "recordedDate": "2021-01-15",
  "reaction": [
    {
      "substance": {
        "coding": [
          {
            "system": "http://snomed.info/sct",
            "code": "256349002",
            "display": "Peanut"
          }
        ]
      },
      "manifestation": [
        {
          "coding": [
            {
              "system": "http://snomed.info/sct",
              "code": "271807003",
              "display": "Rash"
            }
          ],
          "text": "Rash"
        }
      ],
      "description": "Rash after eating peanuts",
      "onset": "2010-06-12T15:30:00Z",
      "severity": "severe"
    }
  ]
}

invalid_allergy_data = {}

try:
    client.resource('AllergyIntolerance', **valid_allergy_data) \
        .is_valid(raise_exception=True)
except OperationOutcome as e:
    pprint('Error: {}'.format(e))

### 3.2 create AllergyIntolerance
**Action:** FHIR client creates a new AllergyIntolerance instance for a patient and save to FHIR server.

In [ ]:
allergy = client.resource("AllergyIntolerance", **valid_allergy_data)
allergy.create()

In [ ]:
example_allergy = client.reference('AllergyIntolerance', valid_allergy_data["id"]).to_resource()
pprint(example_allergy.serialize())